In [ ]:
! aws s3 cp <path_to_model.tar.gz> .

In [ ]:
! tar -xvf model.tar.gz

In [ ]:
! pip install deepspeed

In [ ]:
! pip install transformers sentencepiece

In [ ]:
import torch
from sharded_data_parallel_checkpoint import get_full_state_dict_from_sharded_data_parallel_checkpoint

In [ ]:
full_model = get_full_state_dict_from_sharded_data_parallel_checkpoint(".", tag=f"gptneo20b_instruct_finetuning_1_partial",dtype=torch.float32)


In [ ]:
from transformers import (
    CONFIG_MAPPING,
    MODEL_MAPPING,
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer
)

model_name = "EleutherAI/gpt-j-6b"

config = AutoConfig.from_pretrained(model_name)

In [ ]:
model = AutoModelForSeq2SeqLM.from_config(
            config=config
        )

In [ ]:
model.load_state_dict(full_model,strict=False)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(".")

In [ ]:
input_text = "<sample text>"


In [ ]:
def generate_response(text:str,min_length:str=20,max_new_tokens:str=50,temperature:float=1.0):
    instruction = "Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: "
    response_prefix = "### Response: "
    input = instruction + text + response_prefix
    inputs = tokenizer(input,return_tensors="pt")
    outputs = model.generate(**inputs, early_stopping = True,
          length_penalty = 2.0,
          max_new_tokens = max_new_tokens,
          temperature =  temperature,
          min_length = min_length,
          no_repeat_ngram_size = 2)
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    if response.startswith("Response: "):
        response = response.replace("Response: ","")
    
    return response

In [ ]:
print(generate_response(input_text))